## Step 1 Import Lib

In [3]:
#to install
#!pip install easyocr
!pip install opencv-python
#!pip install imutils

In [1]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import imutils
import easyocr

In [2]:
def read_plat(images):
    img = cv2.imread(images)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #Grayscale and Blur
    #Apply filter and find edges for localization
    bfilter = cv2.bilateralFilter(gray, 11, 17, 17) #Noise reduction
    edged = cv2.Canny(bfilter, 30, 200) #Edge detection
    # Find Contours and Apply Mask
    keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(keypoints)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]
    
    location = None
    for contour in contours:
        approx = cv2.approxPolyDP(contour, 10, True)
        if len(approx) == 4:
            location = approx
            break
            
    mask = np.zeros(gray.shape, np.uint8)
    new_image = cv2.drawContours(mask, [location], 0,255, -1)
    new_image = cv2.bitwise_and(img, img, mask=mask)
    
    (x,y) = np.where(mask==255)
    (x1, y1) = (np.min(x), np.min(y))
    (x2, y2) = (np.max(x), np.max(y))
    cropped_image = gray[x1:x2+1, y1:y2+1]
    
    #use easy ocr to read text
    reader = easyocr.Reader(['en'])
    result = reader.readtext(cropped_image)
    text = result[0][-2]
    return text

In [ ]:
import cv2
##plis yg ini disesuaiin sama path filenya
harcascade = "model/haarcascade_russian_plate_number.xml"

cap = cv2.VideoCapture(0)

cap.set(3, 640) # width
cap.set(4, 480) #height

min_area = 500
count = 0

while True:
    success, img = cap.read()

    plate_cascade = cv2.CascadeClassifier(harcascade)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    plates = plate_cascade.detectMultiScale(img_gray, 1.1, 4)

    for (x,y,w,h) in plates:
        area = w * h

        if area > min_area:
            cv2.rectangle(img, (x,y), (x+w, y+h), (0,255,0), 2)
            cv2.putText(img, "Number Plate", (x,y-5), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (255, 0, 255), 2)

            img_roi = img[y: y+h, x:x+w]
            cv2.imshow("ROI", img_roi)
textplat = read_plat(img)
print(textplat)    

In [ ]:
def save_results(text, csv_filename):
    with open(csv_filename, mode='a', newline='') as f:
        csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerow([text])

In [3]:
import cv2
import numpy as np
import imutils
import easyocr

harcascade = "model/haarcascade_russian_plate_number.xml"

cap = cv2.VideoCapture(0)

cap.set(3, 640) # width
cap.set(4, 480) #height

min_area = 500
count = 0

while True:
    success, img = cap.read()

    plate_cascade = cv2.CascadeClassifier(harcascade)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    plates = plate_cascade.detectMultiScale(img_gray, 1.1, 4)

    for (x, y, w, h) in plates:
        area = w * h

        if area > min_area:
            cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(img, "Number Plate", (x, y-5), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (255, 0, 255), 2)

            img_roi = img[y: y+h, x:x+w]
            cv2.imshow("ROI", img_roi)

            # Mengirim gambar yang terdeteksi ke fungsi read_plat
            plate_text = read_plat(img_roi)
            print("Plat nomor berhasil terdeteksi:", plate_text)

            # Disini bisa ditambahkan logika untuk menyimpan gambar ke lokal jika diperlukan

            # Menghentikan pengambilan gambar setelah deteksi pertama
            break

    cv2.imshow("Result", img)

    if cv2.waitKey(1) & 0xFF == ord('s'):
        if plate_text:  # Pastikan plat nomor sudah terdeteksi sebelum menyimpan gambar
            cv2.imwrite("plates/scaned_img_" + str(count) + ".jpg", img_roi)
            cv2.rectangle(img, (0, 200), (640, 300), (0, 255, 0), cv2.FILLED)
            cv2.putText(img, "Plate Saved", (150, 265), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2, (0, 0, 255), 2)
            cv2.imshow("Results", img)
            cv2.waitKey(500)
            count += 1

        # Memberhentikan penangkapan setelah menyimpan gambar
        break

# Setelah selesai, lepaskan kamera dan tutup jendela OpenCV
cap.release()
cv2.destroyAllWindows()


TypeError: Can't convert object to 'str' for 'filename'

In [2]:
import cv2

harcascade = "model/haarcascade_russian_plate_number.xml"

cap = cv2.VideoCapture(0)

cap.set(3, 640)  # width
cap.set(4, 480)  # height

min_area = 500
count = 0

while True:
    success, img = cap.read()

    plate_cascade = cv2.CascadeClassifier(harcascade)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    plates = plate_cascade.detectMultiScale(img_gray, 1.1, 4)

    for (x, y, w, h) in plates:
        area = w * h

        if area > min_area:
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(img, "Number Plate Detected", (x, y - 5), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (255, 0, 255),
                        2)

            img_roi = img[y: y + h, x:x + w]
            cv2.imshow("ROI", img_roi)

            # Save the detected plate image
            cv2.imwrite("plates/scaned_img_" + str(count) + ".jpg", img_roi)
            count += 1

            # Show success message for a few seconds
            cv2.rectangle(img, (0, 200), (640, 300), (0, 255, 0), cv2.FILLED)
            cv2.putText(img, "Plate Saved", (150, 265), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2, (0, 0, 255), 2)
            cv2.imshow("Results", img)
            cv2.waitKey(2000)  # Show the success message for 2 seconds

            # Close the camera
            cap.release()
            cv2.destroyAllWindows()
            exit()

    cv2.imshow("Result", img)

    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to quit the loop
        break

# Release the camera and close all windows when 'q' is pressed
cap.release()
cv2.destroyAllWindows()


error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [ ]:
import cv2

harcascade = "model/haarcascade_russian_plate_number.xml"

cap = cv2.VideoCapture(0)

cap.set(3, 640)  # width
cap.set(4, 480)  # height

min_area = 500
count = 0

while True:
    success, img = cap.read()

    if not success:
        print("Failed to read frame. Exiting...")
        break

    plate_cascade = cv2.CascadeClassifier(harcascade)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    plates = plate_cascade.detectMultiScale(img_gray, 1.1, 4)

    for (x, y, w, h) in plates:
        area = w * h

        if area > min_area:
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(img, "Number Plate Detected", (x, y - 5), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (255, 0, 255),
                        2)

            img_roi = img[y: y + h, x:x + w]
            cv2.imshow("ROI", img_roi)

            # Save the detected plate image
            cv2.imwrite("plates/scaned_img_" + str(count) + ".jpg", img_roi)
            count += 1

            # Show success message for a few seconds
            cv2.rectangle(img, (0, 200), (640, 300), (0, 255, 0), cv2.FILLED)
            cv2.putText(img, "Plate Saved", (150, 265), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2, (0, 0, 255), 2)
            cv2.imshow("Results", img)
            cv2.waitKey(2000)  # Show the success message for 2 seconds

            # Close the camera
            cap.release()
            cv2.destroyAllWindows()
            exit()

    cv2.imshow("Result", img)

  


In [1]:
import cv2

harcascade = "model/haarcascade_russian_plate_number.xml"

cap = cv2.VideoCapture(0)

cap.set(3, 640)  # width
cap.set(4, 480)  # height

min_area = 500
count = 0

while True:
    success, img = cap.read()

    if not success:
        print("Failed to read frame. Exiting...")
        break

    plate_cascade = cv2.CascadeClassifier(harcascade)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    plates = plate_cascade.detectMultiScale(img_gray, 1.1, 4)

    for (x, y, w, h) in plates:
        area = w * h

        if area > min_area:
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(img, "Number Plate Detected", (x, y - 5), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (255, 0, 255),
                        2)

            img_roi = img[y: y + h, x:x + w]
            cv2.imshow("ROI", img_roi)

            # Save the detected plate image
            cv2.imwrite("plates/scaned_img_" + str(count) + ".jpg", img_roi)
            count += 1

            # Show success message for a few seconds
            cv2.rectangle(img, (0, 200), (640, 300), (0, 255, 0), cv2.FILLED)
            cv2.putText(img, "Plate Saved", (150, 265), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2, (0, 0, 255), 2)
            cv2.imshow("Results", img)
            cv2.waitKey(2000)  # Show the success message for 2 seconds

            # Close the camera
            cap.release()
            cv2.destroyAllWindows()
            exit()

    cv2.imshow("Result", img)

    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to quit the loop
        break

# Release the camera and close all windows when 'q' is pressed
cap.release()
cv2.destroyAllWindows()


Failed to read frame. Exiting...


In [1]:
import cv2

harcascade = "model/haarcascade_russian_plate_number.xml"

cap = cv2.VideoCapture(0)

cap.set(3, 640)  # width
cap.set(4, 480)  # height

min_area = 500
count = 0

while True:
    success, img = cap.read()

    if not success:
        print("Failed to read frame. Exiting...")
        break

    plate_cascade = cv2.CascadeClassifier(harcascade)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    plates = plate_cascade.detectMultiScale(img_gray, 1.1, 4)

    for (x, y, w, h) in plates:
        area = w * h

        if area > min_area:
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(img, "Number Plate Detected", (x, y - 5), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (255, 0, 255),
                        2)

            img_roi = img[y: y + h, x:x + w]
            cv2.imshow("ROI", img_roi)

            # Save the detected plate image
            cv2.imwrite("detected_plates/scaned_img_" + str(count) + ".jpg", img_roi)
            count += 1

            # Show success message for a few seconds
            cv2.rectangle(img, (0, 200), (640, 300), (0, 255, 0), cv2.FILLED)
            cv2.putText(img, "Plate Saved", (150, 265), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2, (0, 0, 255), 2)
            cv2.imshow("Results", img)
            cv2.waitKey(2000)  # Show the success message for 2 seconds

            # Close the camera
            cap.release()
            cv2.destroyAllWindows()
            exit()

    cv2.imshow("Result", img)

    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to quit the loop
        break

# Release the camera and close all windows when 'q' is pressed
cap.release()
cv2.destroyAllWindows()


Failed to read frame. Exiting...


In [1]:
import cv2

harcascade = "model/haarcascade_russian_plate_number.xml"

cap = cv2.VideoCapture(1)

if not cap.isOpened():
    print("Failed to open camera. Exiting...")
    exit()

cap.set(3, 640)  # width
cap.set(4, 480)  # height

min_area = 500
count = 0

while True:
    success, img = cap.read()

    plate_cascade = cv2.CascadeClassifier(harcascade)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    plates = plate_cascade.detectMultiScale(img_gray, 1.1, 4)

    for (x, y, w, h) in plates:
        area = w * h

        if area > min_area:
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(img, "Number Plate Detected", (x, y - 5), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (255, 0, 255),
                        2)

            img_roi = img[y: y + h, x:x + w]
            cv2.imshow("ROI", img_roi)

            # Save the detected plate image
            cv2.imwrite("detected_plates/scaned_img_" + str(count) + ".jpg", img_roi)
            count += 1

            # Show success message for a few seconds
            cv2.rectangle(img, (0, 200), (640, 300), (0, 255, 0), cv2.FILLED)
            cv2.putText(img, "Plate Saved", (150, 265), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2, (0, 0, 255), 2)
            cv2.imshow("Results", img)
            cv2.waitKey(2000)  # Show the success message for 2 seconds

            # Close the camera
            cap.release()
            cv2.destroyAllWindows()
            exit()

    cv2.imshow("Result", img)

    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to quit the loop
        break

# Release the camera and close all windows when 'q' is pressed
cap.release()
cv2.destroyAllWindows()


Failed to open camera. Exiting...


error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [1]:
import cv2

harcascade = "model/haarcascade_russian_plate_number.xml"

cap = cv2.VideoCapture(0)

cap.set(3, 640)  # width
cap.set(4, 480)  # height

min_area = 500
count = 0

while True:
    success, img = cap.read()

    if not success:
        print("Failed to read frame. Exiting...")
        break

    plate_cascade = cv2.CascadeClassifier(harcascade)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    plates = plate_cascade.detectMultiScale(img_gray, 1.1, 4)

    for (x, y, w, h) in plates:
        area = w * h

        if area > min_area:
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(img, "Number Plate Detected", (x, y - 5), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (255, 0, 255),
                        2)

            img_roi = img[y: y + h, x:x + w]
            cv2.imshow("ROI", img_roi)

            # Save the detected plate image
            cv2.imwrite("detected_plates/scaned_img_" + str(count) + ".jpg", img_roi)
            count += 1

            # Show success message for a few seconds
            cv2.rectangle(img, (0, 200), (640, 300), (0, 255, 0), cv2.FILLED)
            cv2.putText(img, "Plate Saved", (150, 265), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2, (0, 0, 255), 2)
            cv2.imshow("Results", img)
            cv2.waitKey(2000)  # Show the success message for 2 seconds

            # Close the camera
            cap.release()
            cv2.destroyAllWindows()
            exit()

    cv2.imshow("Result", img)

    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to quit the loop
        break

# Release the camera and close all windows when 'q' is pressed
cap.release()
cv2.destroyAllWindows()


Failed to read frame. Exiting...
